In [35]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import defaultdict
import pyfastx
import numpy as np
from sklearn.model_selection import train_test_split
from numpy import array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
import math
from window_slider import Slider
from Bio.Seq import Seq
from complexcgr import FCGR

In [36]:
def make_cgr(x, D=6):

   
    fcgr = FCGR(k=D)
   
    seqs = []
    for seq in x:
   
        tmp = seq.transpose()
        padding = np.sum(tmp, axis=1)
        padding = padding > 0
        
        tmp = tmp[padding, :]
   
        tmp = np.argmax(tmp, axis = 1)
   
        tmp = np.where(tmp == 0, "A", tmp)
        tmp = np.where(tmp == "1", "C", tmp)
        tmp = np.where(tmp == "2", "G", tmp)
        tmp = np.where(tmp == "3", "T", tmp)
        tmp = np.where(tmp == "4", "N", tmp)
   
        tmp = "".join(tmp)
       
        tmp = fcgr(tmp)

        max_sz = 4**D
        fcgr_sum = np.sum(tmp)
       
        tmp = tmp / fcgr_sum
        tmp = tmp * max_sz
   
        seqs.append(tmp)

    seqs = np.asarray(seqs)

    return seqs

In [37]:
gisaid_fa = pyfastx.Fasta('/home/hnguyen/Documents/PhD/IAV_Data/GISAID/data/gisaid.fasta', build_index=True)
# Create seq id dict
gisaid_seq_id_dict = defaultdict()
for seq in gisaid_fa:
    seq_id = seq.name.split('|')[0]
    gisaid_seq_id_dict[seq_id] = seq.name

## Onehot and FCGR encoding for RNA Sequences

In [38]:
nucleotide = ['A', 'C', 'G', 'T']
nc_dict = {}
for i, aa in enumerate(nucleotide):
    nc_dict[aa] = i

In [39]:
def get_onehot_encoding(df, nc_dict, segment_id, max_length, rev_com_id):
    df = df.reset_index(drop=True)
    for index, row in df.iterrows():
        seq_id = gisaid_seq_id_dict[row[segment_id]]
        sequence = Seq(gisaid_fa[seq_id].seq.upper())
        if row[segment_id] in rev_com_id:
            print (row[segment_id])
            sequence = sequence.reverse_complement()
        tmp = np.zeros((1, len(nc_dict)+1, max_length), dtype=np.float32)
        for i, seq in enumerate(sequence):
            if seq == 'A' or seq == 'C' or seq == 'G' or seq == 'T':
                tmp[:, nc_dict[seq], i] = 1.0
            else: # any ambiguous bases
                tmp[:, 4, i] = 1.0
        if index == 0:
            final_array = tmp
        else:
            final_array = np.vstack((final_array, tmp))
    return final_array

In [40]:
def get_onehot_chunk(df, nc_dict, segment_id, max_length, rev_com_id, bucket_size=5000):
    list = np.arange(df.shape[0])
    bucket_size = bucket_size
    overlap_count = 0
    slider = Slider(bucket_size,overlap_count)
    slider.fit(list)   
    one_hot_array = []
    while True:
        window_data = slider.slide()
        tmp_df = df.iloc[window_data]
        print (tmp_df.index)
        one_hot = get_onehot_encoding(tmp_df, nc_dict, segment_id, max_length, rev_com_id)
        one_hot_array.append(one_hot)
        if slider.reached_end_of_list(): break
    return one_hot_array

In [41]:
# Example rev seq
rev_com_id = ['EPI2040903', 'EPI2647172', 'EPI541968', 'EPI24822', 'EPI2097991',
       'EPI2097998', 'EPI2098034', 'EPI2098042', 'EPI2098048',
       'EPI2098071', 'EPI2098091', 'EPI2098098', 'EPI2098104',
       'EPI2098144', 'EPI2098152', 'EPI2098176', 'EPI2098183',
       'EPI2098191', 'EPI2098205', 'EPI2098213', 'EPI2098239',
       'EPI2098247', 'EPI2098263', 'EPI2098287', 'EPI2098299',
       'EPI2098307', 'EPI2098350', 'EPI2098365', 'EPI2195589',
       'EPI2842840', 'EPI2555843', 'EPI2434733', 'EPI2434734',
       'EPI2570421', 'EPI2842207', 'EPI2842251', 'EPI2806788',
       'EPI2841497', 'EPI2841509', 'EPI2545628', 'EPI2545984',
       'EPI2806682', 'EPI2806671']

In [42]:
df_train = pd.read_csv('train.csv')

In [43]:
df_train.shape

(99, 41)

In [44]:
df_train = df_train.reset_index(drop=True)

In [45]:
X_train_onehot_array = get_onehot_chunk(df_train, nc_dict, '4_HA_ID', 2100, rev_com_id, bucket_size=50)

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
      dtype='int64')
Index([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98],
      dtype='int64')


In [46]:
X_train_onehot = np.concatenate(X_train_onehot_array, axis=0)

In [47]:
X_train_onehot.shape

(99, 5, 2100)

In [48]:
X_train_fcgr =  make_cgr(X_train_onehot, 6).astype(np.float32)

In [49]:
X_train_fcgr.shape

(99, 64, 64)

In [50]:
X_train_fcgr[0,:,:]

array([[0.       , 0.       , 0.       , ..., 7.2581215, 4.838748 ,
        9.677496 ],
       [0.       , 0.       , 0.       , ..., 4.838748 , 2.419374 ,
        9.677496 ],
       [0.       , 2.419374 , 0.       , ..., 2.419374 , 0.       ,
        2.419374 ],
       ...,
       [0.       , 0.       , 2.419374 , ..., 0.       , 2.419374 ,
        0.       ],
       [0.       , 2.419374 , 2.419374 , ..., 2.419374 , 0.       ,
        0.       ],
       [2.419374 , 2.419374 , 2.419374 , ..., 2.419374 , 4.838748 ,
        0.       ]], dtype=float32)

## Onehot encoding for protein sequence 

In [51]:
aminos = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
aa_dict = {}
for i, aa in enumerate(aminos):
    aa_dict[aa] = i

In [52]:
max_length = 585
def get_onehot_encoding(df):
    df = df.reset_index(drop=True)
    for index, row in df.iterrows():
        seq_id = gisaid_seq_id_dict[row['4_HA_ID']]
        sequence = gisaid_protein_fa[seq_id].seq.upper()
        tmp = np.zeros((1, len(aa_dict)+1, max_length), dtype=np.float32)
        for i, seq in enumerate(sequence):
            if seq in aminos:
                tmp[:, aa_dict[seq], i] = 1.0
            else: # any ambiguous bases
                tmp[:, 20, i] = 1.0
        if index == 0:
            final_array = tmp
        else:
            final_array = np.vstack((final_array, tmp))
    return final_array

In [53]:
def get_oneot_chunk(df, bucket_size=5000):
    list = np.arange(df.shape[0])
    bucket_size = bucket_size
    overlap_count = 0
    slider = Slider(bucket_size,overlap_count)
    slider.fit(list)   
    one_hot_array = []
    while True:
        window_data = slider.slide()
        tmp_df = df.iloc[window_data]
        print (tmp_df.index)
        one_hot = get_onehot_encoding(tmp_df)
        one_hot_array.append(one_hot)
        if slider.reached_end_of_list(): break
    return one_hot_array